In [4]:
import numpy as np
import pandas as pd
import itertools

from scipy import stats
from statsmodels.stats.descriptivestats import sign_test
from statsmodels.stats.weightstats import zconfint

In [1]:
data=[49,58,75,110,112,132,151,276,281,362]

In [2]:
data

[49, 58, 75, 110, 112, 132, 151, 276, 281, 362]

In [3]:
data[0]


49

In [10]:
stats.wilcoxon([x - 200. for x in data])

WilcoxonResult(statistic=17.0, pvalue=0.28450269791120752)

In [8]:
[x - 200 for x in data]

[-151, -142, -125, -90, -88, -68, -49, 76, 81, 162]

In [11]:
non_cut=[22,22,15,13,19,19,18,20,21,13,13,15]
cut=[17,18,18,15,12,4,14,15,10]

In [14]:
5./7

0.7142857142857143

In [20]:
stats.mannwhitneyu(non_cut,cut, use_continuity=True, alternative='greater')

MannwhitneyuResult(statistic=81.0, pvalue=0.029004992720873729)

In [22]:
data = pd.read_csv('challenger.txt', sep = '\t')

In [23]:
data.head()

,Unnamed: 0,Temperature,Incident
0,Apr12.81,18.9,0
1,Nov12.81,21.1,1
2,Mar22.82,20.6,0
3,Nov11.82,20.0,0
4,Apr04.83,19.4,0


In [25]:
len(data)

23

In [24]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

In [26]:
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [32]:
t0 = data[data.Incident == 0].Temperature.values
t1 = data[data.Incident == 1].Temperature.values

In [33]:
t0

array([ 18.9,  20.6,  20. ,  19.4,  22.2,  22.8,  21.1,  25.6,  19.4,
        19.4,  23.9,  21.1,  27.2,  24.4,  26.1,  24.4])

In [65]:
np.random.seed(0)

t0_mean_scores = map(np.mean, get_bootstrap_samples(t0, 1000))
#print "95% confidence interval for t0:",  stat_intervals(t0_mean_scores, 0.05)

t1_mean_scores = map(np.mean, get_bootstrap_samples(t1, 1000))
#print "95% confidence interval for t1:",  stat_intervals(t1_mean_scores, 0.05)

In [66]:
delta_mean_scores = map(lambda x: x[0] - x[1], zip(t0_mean_scores, t1_mean_scores))

In [67]:
print "95% confidence interval for the difference between medians",  stat_intervals(delta_mean_scores, 0.05)

95% confidence interval for the difference between medians [ 1.42299107  7.93861607]


In [63]:
round(1.42299107,4)

1.423

In [37]:
def permutation_t_stat_ind(sample1, sample2):
    return np.mean(sample1) - np.mean(sample2)

In [38]:
def get_random_combinations(n1, n2, max_combinations):
    index = range(n1 + n2)
    indices = set([tuple(index)])
    for i in range(max_combinations - 1):
        np.random.shuffle(index)
        indices.add(tuple(index))
    return [(index[:n1], index[n1:]) for index in indices]

In [39]:
def permutation_zero_dist_ind(sample1, sample2, max_combinations = None):
    joined_sample = np.hstack((sample1, sample2))
    n1 = len(sample1)
    n = len(joined_sample)
    
    if max_combinations:
        indices = get_random_combinations(n1, len(sample2), max_combinations)
    else:
        indices = [(list(index), filter(lambda i: i not in index, range(n))) \
                    for index in itertools.combinations(range(n), n1)]
    
    distr = [joined_sample[list(i[0])].mean() - joined_sample[list(i[1])].mean() \
             for i in indices]
    return distr

In [40]:
def permutation_test(sample, mean, max_permutations = None, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    t_stat = permutation_t_stat_ind(sample, mean)
    
    zero_distr = permutation_zero_dist_ind(sample, mean, max_permutations)
    
    if alternative == 'two-sided':
        return sum([1. if abs(x) >= abs(t_stat) else 0. for x in zero_distr]) / len(zero_distr)
    
    if alternative == 'less':
        return sum([1. if x <= t_stat else 0. for x in zero_distr]) / len(zero_distr)

    if alternative == 'greater':
        return sum([1. if x >= t_stat else 0. for x in zero_distr]) / len(zero_distr)

In [41]:
np.random.seed(0)
print "p-value: %f" % permutation_test(t0, t1, max_permutations = 10000)

p-value: 0.007000
